In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
import pyspiel
import open_spiel.python.games
from open_spiel.python.observation import make_observation
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
from open_spiel.python.examples.ppo_utils import EnvParams
from open_spiel.python.examples.ppo_eval import eval_agents_parallel
from open_spiel.python.env_decorator import *


from auctions.webutils import *
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
import numpy as np 
import pandas as pd
from absl import logging
import tempfile
import subprocess

import open_spiel.python.games
from open_spiel.python.games.clock_auction_base import *
from open_spiel.python.examples.ubc_utils import *

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from django.db.models import F
from auctions.models import *


os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import itertools
from open_spiel.python.examples.ubc_cma import *
from open_spiel.python.examples.env_and_policy import *
from open_spiel.python.examples.ppo_eval import run_eval
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
import copy
import sys
sys.path.append('/apps/sats/python')
from pysats import map_generators, run_sats

from auctions.webutils import convert_pesky_np

from open_spiel.python.algorithms import get_all_states_with_policy
import time
import tempfile
import copy
import pickle

from open_spiel.python.algorithms.outcome_sampling_mccfr import OutcomeSamplingSolver
from open_spiel.python.algorithms.external_sampling_mccfr import ExternalSamplingSolver
import signal


In [4]:
g = Game.objects.last().load_as_spiel()
o1 = g.make_py_observer(params={'normalize': False})
o2 = g.make_py_observer()

o2.set_from(g.new_initial_state().child(0).child(0).child(1).child(1), 0)
o2.tensor

array([0.1015625 , 0.1015625 , 0.1015625 , ..., 0.3281977 , 0.33430234,
       0.38343024], dtype=float32)

In [5]:
o1.set_from(g.new_initial_state().child(0).child(0).child(1).child(1), 0)
o1.tensor

array([ 13. ,  13. ,  13. , ..., 112.9, 115. , 131.9], dtype=float32)

In [ ]:
PREFIX = 'sep19_encumbered'
CONFIG_DIR = os.environ['CLOCK_AUCTION_CONFIG_DIR']
PYSATS = '/apps/open_spiel/open_spiel/python/examples'

In [ ]:
configs = []


# This is the old 2 player games you know and love
# with open('/apps/open_spiel/open_spiel/python/examples/oq4.pkl', 'rb') as f:

# These are new 2 player games from later in the summer
# with open('/apps/open_spiel/open_spiel/python/examples/oq5.pkl', 'rb') as f:
#     configs += pickle.load(f)


with open('/apps/open_spiel/open_spiel/python/examples/sep18_encumbered.pkl', 'rb') as f:
    configs += pickle.load(f)

# with open('/apps/open_spiel/open_spiel/python/examples/sep18_3province.pkl', 'rb') as f:
#     configs += pickle.load(f)


# These are 3 player games
# with open('/apps/open_spiel/open_spiel/python/examples/oq7.pkl', 'rb') as f:
#     configs += pickle.load(f)
# with open('/apps/open_spiel/open_spiel/python/examples/oq6.pkl', 'rb') as f:
#     configs += pickle.load(f)
print(len(configs))
configs

In [ ]:
# Generate a mod for each game
all_mods = defaultdict()
for i, settings in enumerate(configs):
    mods = defaultdict()
    
    ####
    settings['auction_params']['agent_memory'] = 10
    settings['auction_params']['max_rounds'] = 10
    ####
    
    # First, do the valuation "bases"
    
    mods['base'] = settings

    # Spite
    spite = copy.deepcopy(settings)
    spite['pricing_bonus'] = 0.05
    mods['spite'] = spite 

    # Default with risk aversion 
    risk_averse = copy.deepcopy(settings)
    risk_averse['risk_averse'] = True
    mods['risk_averse'] = risk_averse

    for valuation in ['base', 'risk_averse', 'spite']:

        # Change to hide demand
        hide_demand = copy.deepcopy(mods[valuation])
        hide_demand['auction_params']['information_policy'] = 'hide_demand'
        mods[f'{valuation}_hide_demand'] = hide_demand

        # Change clock speed
        high_speed = copy.deepcopy(mods[valuation])
        high_speed['auction_params']['increment'] *= 2
        mods[f'{valuation}_high_speed'] = high_speed

        # Change clock speed again
        medium_speed = copy.deepcopy(mods[valuation])
        medium_speed['auction_params']['increment'] = float(np.round(1.5 * medium_speed['auction_params']['increment'], 2))
        mods[f'{valuation}_medium_speed'] = medium_speed

        # Sanity check: allow undersell
        undersell_allowed = copy.deepcopy(mods[valuation])
        undersell_allowed['auction_params']['undersell_policy'] = UndersellPolicy.UNDERSELL_ALLOWED.name
        mods[f'{valuation}_undersell_allowed'] = undersell_allowed

        # Sanity check: activity doesn't matter
        no_activity = copy.deepcopy(mods[valuation])
        no_activity['auction_params']['activity_policy'] = ActivityPolicy.OFF.name
        mods[f'{valuation}_no_activity'] = no_activity

        # Change the tie-breaking method
        tie_break = copy.deepcopy(mods[valuation])
        tie_break['auction_params']['tiebreaking_policy'] = TiebreakingPolicy.DROP_BY_LICENSE.name
        mods[f'{valuation}_tie_break'] = tie_break

        # Grace period
        grace_period = copy.deepcopy(mods[valuation])
        grace_period['auction_params']['grace_rounds'] = 2
        mods[f'{valuation}_grace_period'] = grace_period

        # TODO: Signal, Greg's signal idea

        # # Mod 3 imperfect base
        # imperfect = copy.deepcopy(settings)
        # imperfect['auction_params']['agent_memory'] = 2
        # mods['imperfect'] = imperfect

    all_mods[i] = mods

In [ ]:
OUTPUT_DIR = f'{PYSATS}/{PREFIX}'
!mkdir -p {OUTPUT_DIR}
!rm -rf {CONFIG_DIR}/{PREFIX}
!mkdir -p {CONFIG_DIR}/{PREFIX}

In [ ]:
# Write out YML files for the modified games
for base_name, game_mods in tqdm(all_mods.items()):
    for mod_name, mod in game_mods.items():
        g_name = f'{base_name}_{mod_name}'
        outfile_name = f'{OUTPUT_DIR}/{g_name}.yml'
        with open(outfile_name, 'w') as f:
            yaml.dump(convert_pesky_np(mod), f)
        # Run SATS
        command = f'{PYSATS}/pysats.py --config_file {outfile_name} --seed {mod["sats_seed"]} --output_file {CONFIG_DIR}/{PREFIX}/{PREFIX}_{g_name}.json'
        !python {command}